In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
TEIS = pd.read_excel(r"C:\Users\mnorm\Documents\NSS\Python\Projects\ddid-rasberry-pi\data\TEIS Exit Data for NSS.xlsx", sheet_name = 3)

In [ ]:
TEIS_subset = TEIS[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date','Days from I-IFSP to Exit', 'Exit Note']]
TEIS_subset

### Removing Rows with Null Values in the Exit Note Column

In [ ]:
TEIS_filt = TEIS_subset[TEIS_subset['Exit Note'].notnull()]
pd.set_option('display.max_colwidth', None)
TEIS_filt

### Filtering for Exit Notes that contain Dates

In [ ]:
TEIS_with_dates = TEIS_filt.loc[TEIS_filt['Exit Note'].str.contains(r"(\d+/\d+/\d+)", case=False)]
TEIS_with_dates

### Filtering for Exit Notes that contain the phrase 'Reopened'

In [ ]:
TEIS_reopened = TEIS_with_dates.loc[TEIS_with_dates['Exit Note'].str.contains('reopened[a-z]*', flags=re.I, regex=True)]
TEIS_reopened

## Testing Regex

In [ ]:
z = TEIS_reopened.loc[ 5, 'Exit Note']
z

In [ ]:
#isolating the dates
date = re.findall(r'\d{2}/\d{2}/\d{4}', z)

for dates in date :
    print(dates)

In [ ]:
#isolating the string/date column to confirm I'm pulling the desired dates
string = re.findall(r'Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4}', z)
string

In [ ]:
y = TEIS_reopened.loc[30, 'Exit Note']
y

In [ ]:
#ensuring that this expression will extract every occurance of this pattern within the string
re.findall(r'Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4}', y)

In [ ]:
#splitting the pattern by Reopened/Exit Date so that they can go in separate columns
w = re.findall(r'Reopened: \d{2}/\d{2}/\d{4}', y)
v = re.findall(r'Exit Date: \d{2}/\d{2}/\d{4}', y)

In [ ]:
ww = " ".join(map(str,w))
type(ww)

In [ ]:
#extracting just the date from the reopened/exit date string
re.findall(r'\d{2}/\d{2}/\d{4}', ww)

In [ ]:
#try creating a for loop to extract the dates if converting to a string doesn't work
re.findall(r'\d{2}/\d{2}/\d{4}', w[0])

## Using Regex on DataFrame

In [ ]:
x = TEIS_reopened[['Exit Note']]
x

In [ ]:
dates = TEIS_reopened['Exit Note'].str.findall(r'\d{2}/\d{2}/\d{4}')
dates

In [ ]:
#string_col = TEIS_reopened['Exit Note'].str.findall(r'Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4}')
#string_col

In [ ]:
string_col = TEIS_reopened['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4} Exit Date: \d{2}/\d{2}/\d{4})')
string_col

In [ ]:
#reopened_dates = TEIS_reopened['Exit Note'].str.findall(r'Reopened: \d{2}/\d{2}/\d{4}')
#reopened_dates  

In [ ]:
reopened_dates = TEIS_reopened['Exit Note'].str.extract(r'(Reopened: \d{2}/\d{2}/\d{4})')
reopened_dates           

In [ ]:
#exit_dates = TEIS_reopened['Exit Note'].str.findall(r'Exit Date: \d{2}/\d{2}/\d{4}')
#exit_dates

In [ ]:
exit_dates = TEIS_reopened['Exit Note'].str.extract(r'(Exit Date: \d{2}/\d{2}/\d{4})')
exit_dates

**Creating New Column for Reopened Dates** 

In [ ]:
#TEIS_reopened['Reopened Dates'] = TEIS_reopened['Exit Note'].str.findall(r'Reopened: \d{2}/\d{2}/\d{4}')

In [ ]:
TEIS_reopened['Reopened Dates'] = TEIS_reopened['Exit Note'].str.extract(r'Reopened: (\d{2}/\d{2}/\d{4})')

**Creating New Column for Exit Dates** 

In [ ]:
#TEIS_reopened['Exit Dates'] = TEIS_reopened['Exit Note'].str.findall(r'Exit Date: \d{2}/\d{2}/\d{4}')

In [ ]:
TEIS_reopened['Exit Dates'] = TEIS_reopened['Exit Note'].str.extract(r'Exit Date: (\d{2}/\d{2}/\d{4})')

In [ ]:
TEIS_reopened

**Converting to Datetime**

In [ ]:
TEIS_reopened.dtypes

In [ ]:
TEIS_reopened['Reopened Dates'] = pd.to_datetime(TEIS_reopened['Reopened Dates'])

In [ ]:
TEIS_reopened.dtypes

In [ ]:
TEIS_reopened['Exit Dates'] = pd.to_datetime(TEIS_reopened['Exit Dates'])

In [ ]:
TEIS_reopened.dtypes

**Subtracting the Dates**

In [ ]:
TEIS_reopened['Time Elapsed(Days)'] = (TEIS_reopened['Reopened Dates'] - TEIS_reopened['Exit Dates']).dt.days

In [ ]:
TEIS_reopened

In [ ]:
TEIS_reopened.dtypes

In [ ]:
TEIS_reopened['Real IFSP to Exit Time (Days)'] = (TEIS_reopened['Days from I-IFSP to Exit'] - TEIS_reopened['Time Elapsed(Days)'])

In [ ]:
TEIS_reopened = TEIS_reopened.drop(columns = 'Real IFSP to Exit Time')

In [ ]:
TEIS_reopened

REATTEMPT LATER

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.Series.iteritems.html
#pattern = re.findall(r'(?<=Reopened:)\d{2}/\d{2}/\d{4}')
#((?<=Reopened: )|(?<=Exit Date: ))

In [ ]:
#try to get just the reopened and exit dates [COMPLETE]
#look into doing a left join between reopened_dates and exit_dates dataframes just to do the math with extractall as regex method
#figure out if you can subtract them from each other